<h2> Unsupervised Pretraining </h2> 

When we do not have much labeled data, but we cannot find a model trained on a similar task. 

If the data you have is unlabelled, you may still be able to perform unsupervised pretraining on it. 

If you can gather plenty of unlabeled training data, you can try to train the layers one by one starting with the lowest layer and going up. 

<h4> Boltzmaan Machine </h4> 

Instead of using a deterministic step function to decide which value to output, these neurons output 1 with some probability, and 0 otherwise.

If we keep a Boltzmaan machine running long enough, the probbility of observing a perticular outcome will only be a function of the connection weights and bias terms, not of the original configuration.

Training a Boltzmann machine means finding the parameters which make the network approximate the training sets probability distribution.

The point where your model no longer depends on its intital state, it has reached thermal equilibrium where there is no thermal gradient to cause any change. This is a generative model.

<h4> Restricted Boltzmaan Machines </h4> 

RBMs are shallow neural networks with only 2 layers and is an unsupervised training method used to find patterns in data by reconstructing the input. It is called restricted because neurons in the same layer are not connected. 

RBMs can be used for both:
- reconstruction of input
- classify instances lacking classification by using the concept that similar neurons will fire in similar inputs. 

Given that you have more unlabelled data, you start off by building your network using only unlabelled data such that your input in the forward pass with be equal to the backward output. Then after training sufficient hidden layers, you feed in labelled data to train the output layer. 


Currently available methods for us to speed up training are:
- Initialization strategy
- Good activation function
- Batch Normalization
- Reusing Pretrained network

<h2> Faster Optimizers </h2> 

<h2> Momentum Optimization </h2> 

It starts out slow and starts increasing speed as it picks up momentum, as it goes down the gradient slope. 

In gradient descent, it simply subtracts the existing theta values by the current gradient. It has no idea of the earlier gradient of the previous step. 

Momentum optimization subtracts the local gradient from the momentum vector(multiplied by m) and it updates the weights by simply adding this momentum vector. 

The gradient vector thats multiplied with the learning rate is used to control the accelerating here. not the speed. Hence a larger negative gradient results in a higher accelaration. 


Momentum optimization can update weights 10 times faster for a typical beta value of 0.9. This means it will not be stuck in plateaus when reaching global minima, hence it can work with non normalized data faster too. 

momentum <= friction * momentum - lr * gradient
weights <= weights + momentum 

Notice we have added a  new parameter to tune in momentum

It is better to think of Momentum optimization with movng averages. 

moving average of gradient <= momentum * moving average of gradient + (1- momentum) * gradient of this batch

Then the moving average of gradient it used to update the weights:

weights <= weights - (alpha * moving average of gradient)

This ensures that we smoothen out any large movements from single step gradient descents. 

<h3> Nesterov Accelerated Gradient </h3> 

Involves measuring the gradient at a few steps ahead of the actual *theta* value we are currently at. Then we multiply this with the learning rate and subtract from momentum vector * friction. 

This works because in general the momentum vector will be pointing in the right direction. hence to accelerate it a bit, it makes sense to measure the gradient a bit further along the momentum vector. 

Given that we are moving along the momentum vector in finding the gradient, there is less oscillations and it will converge faster. 

<h3> AdaGrad </h3> 

When gradient descent occurs, its quickly starts going down the steepest slope and then slows down at the bottom of the valley. If we can initially make it travel towards the global optimum, it would be better.

In AdaGrad we square the gradients into a vector s by doing element wise multiplication of the gradient vector.

Given that the gradient vector consists of gradients with respect to multiple theta dimensions, if the gradient is steep across one particular theta dimension, the squaring of it will make it more steeper allowing it to travel down the slope faster. 

<h3> RMS Prop (Root Mean Square Prop) </h3> 

RMS Prop is very similar to AdaGrad, however it uses the moving average for each minibatch to update its *s* value. 

In both AdaGrad and RMS Prop, we look to smoothen out the gradient descent process. For dimensions with large oscillations, the elementwise squaring in the first step of the algorithm causes the *s* to become large for dimensions with large oscillations, and smaller for dimensions with small oscillations. 

Hence in the 2nd step of the algorithm, when we divide the gradient by the **s** value for a particular dimension, large oscillations are damped down, and the smaller more important oscillations are magnified, leading to faster optimizations. 

<h3> Adam and Nadam Optimization </h3> 

Adam: Adaptive Moment Estimation combines momentum optimization and RMS Prop. 

Just like momentum, it keeps track of exponentially decaying average of past gradients, and like RMSProp, it keeps track of exponentially decaying average of past squared gradients. 


The momentum parameter for decay of gradient is initialized to 0.9, while the hyperparameter for decay of square of gradient is initialized to 0.999. 

Nadam : This is Adam optimizer with the Nesterov trick of finding the gradient a few steps further along the line of momentum.



<h3> Learning rate scheduling </h3> 

Instead of starting with a large learning rate and dividing it by 3, we can start with a high learning rate, and reduce it once it stops making fast progress. These strategies are called learning schedules. 

Power Scheduling: 
Here we set the learning rate to reduce at each iteration. The function of learning rate will be equal to the iteration number. 

Exponent scheduling:
Set the learning rate to exponential decay of the initial learning rate, where the nate gets decreased by a factor of 10 every *s* steps. 
(Power Scheduling decreases the learning rate much more slowly as compared to Exponent scheduling)

Piecewise constant scheduling:
Here we pre set learning rates for ranges of epochs. This is difficulat as we need to find the appropriate number of epochs to spread a learning rate through. 

Performance scheduling:



